In [1]:
from dataloader import LeukemiaLoader
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
print(torch.cuda.is_available())
device=torch.device('cuda',0)
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.optim.lr_scheduler import StepLR

True


In [2]:
class Res18_basic_Block(nn.Module):
    def __init__(self, inchannel, outchannel, stride = 1):
        super(Res18_basic_Block, self).__init__()
        
        self.basic = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(outchannel)
        )
        
        self.shortcut = nn.Sequential()
        
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        
        out = self.basic(x)
        out = out + self.shortcut(x)
        out = nn.ReLU()(out)
        
        return out
        

In [3]:
import torch.nn as nn

class Bottleneck(nn.Module):
    
    def __init__(self, inchannel, outchannel, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.expansion = 4
        self.basic = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=1),  
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),  
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=stride, padding=1),  
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel * self.expansion, kernel_size=1, stride=1),   
            nn.BatchNorm2d(outchannel * self.expansion)
        )
         
        self.downsample = downsample
        
    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
    
        out = self.basic(x)

        out += identity
        out = nn.ReLU()(out)

        return out


In [4]:
class ResNet_18(nn.Module):
    def __init__(self):
        super(ResNet_18, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.layer1 = self.make_layer(64, 2, stride=1)
        self.layer2 = self.make_layer(128, 2, stride=2)
        self.layer3 = self.make_layer(256, 2, stride=2)        
        self.layer4 = self.make_layer(512, 2, stride=2)  
        self.classify = nn.Linear(512, 2)
            
       
    def make_layer(self, channel, num_blocks, stride):
        
        strides = [stride] + [1]
        layers = []
        for stride in strides:
            layers.append(Res18_basic_Block(self.inchannel, channel, stride))
            self.inchannel = channel
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)
        return out

In [5]:
class ResNet_50(nn.Module):
    def __init__(self):
        super(ResNet_50, self).__init__()
        self.inchannel = 64
        self.expansion = 4
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.num = [3,4,6,3]
        self.layer1 = self.make_layer(64, self.num[0], stride=1)
        self.layer2 = self.make_layer(128, self.num[1], stride=2)
        self.layer3 = self.make_layer(256, self.num[2], stride=2)        
        self.layer4 = self.make_layer(512, self.num[3], stride=2)  
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.classify = nn.Linear(512*4, 2)
            
       
    def make_layer(self,channel, num_blocks, stride):
        
        downsample = None
        if stride != 1 or self.inchannel != channel * self.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inchannel, channel * self.expansion, 1,stride),
                nn.BatchNorm2d(channel * self.expansion)
            )
            
        layers = []
        layers.append(Bottleneck(self.inchannel, channel, downsample=downsample, stride=stride)) 
        self.inchannel = channel*self.expansion   

        for _ in range(1, num_blocks):  
            layers.append(Bottleneck(self.inchannel,channel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)
        return out

In [6]:
class ResNet_152(nn.Module):
    def __init__(self):
        super(ResNet_152, self).__init__()
        self.inchannel = 64
        self.expansion = 4
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.num = [3,8,36,3]
        self.layer1 = self.make_layer(64, self.num[0], stride=1)
        self.layer2 = self.make_layer(128, self.num[1], stride=2)
        self.layer3 = self.make_layer(256, self.num[2], stride=2)        
        self.layer4 = self.make_layer(512, self.num[3], stride=2)  
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.classify = nn.Linear(512*4, 2)
            
       
    def make_layer(self,channel, num_blocks, stride):
        
        downsample = None
        if stride != 1 or self.inchannel != channel * self.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inchannel, channel * self.expansion, 1,stride),
                nn.BatchNorm2d(channel * self.expansion)
            )
            
        layers = []
        layers.append(Bottleneck(self.inchannel, channel, downsample=downsample, stride=stride)) 
        self.inchannel = channel*self.expansion   

        for _ in range(1, num_blocks):  
            layers.append(Bottleneck(self.inchannel,channel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)
        return out

In [7]:
def train(model, loader_train, optimizer, loss_fn):
    model.train()
    total_loss = 0
    total_correct = 0
    counter = 0
    for idx,(data,label, path) in enumerate(loader_train):
        data = data.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.long)

        optimizer.zero_grad()
        pred_y = model(data)
        mono_loss = loss_fn(pred_y, label)
        mono_loss.backward()
        optimizer.step()

        total_loss += mono_loss
        mono_acc = evaluate(pred_y, label)
        total_correct += mono_acc
        counter+=1

    train_loss = total_loss / len(loader_train.dataset)
    train_accuracy = total_correct / counter
    return train_loss, train_accuracy

def test(model, loader_test, loss_fn):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    counter = 0
    with torch.no_grad():
        for idx,(data,label, path) in enumerate(loader_test):
            data = data.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.long)

            pred_y = model(data)
            mono_loss = loss_fn(pred_y, label)

            total_loss += mono_loss
            mono_acc = evaluate(pred_y, label)
            total_correct += mono_acc
            counter+=1

    test_loss = total_loss / len(loader_test.dataset)
    test_accuracy = total_correct / counter
    return test_loss, test_accuracy



In [8]:
def train_and_test(model, loader_train,loader_test): 
    model = model()
    model.to(device)
    lr = 1e-2
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.RAdam(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=3, gamma=0.5)

    total_train = [0]
    total_test = [0]
    epochs = 100

    for epoch in range(1, epochs + 1):
        train_loss, train_accuracy = train(model, loader_train, optimizer, loss_fn)
        total_train.append(train_accuracy)
        test_loss, test_accuracy = test(model, loader_test, loss_fn)
        total_test.append(test_accuracy)
        if(epoch%10 == 0):
            print(f"Epoch {epoch}")
            print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
            print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

        scheduler.step()
    
    return total_train, total_test

In [9]:
def evaluate(pred_y, label):
    pred_y = pred_y.to(device)
    label = label.to(device)
    correct = pred_y.max(dim=1)[1].eq(label).sum().item()
    total = label.shape[0]
    accuracy = (correct / total) * 100
    return accuracy

def save_result(csv_path, predict_result):
    df = pd.read_csv(csv_path)
    new_df = pd.DataFrame()
    new_df['ID'] = df['Path']
    new_df["label"] = predict_result
    new_df.to_csv("./your_student_id_resnet18.csv", index=False)
def plot(resnet_train,resnet_test):
    epoch_list = []
    for i in range(len(resnet_train)):
        epoch_list.append(i)

    plt.plot(epoch_list, resnet_train, label= "res_train")
    plt.plot(epoch_list, resnet_test, label= "res_test")
    plt.ylim(50, 100)
    plt.legend(loc = 4)
    plt.show()

In [10]:
from torch.utils.data import DataLoader, SubsetRandomSampler
if __name__ == "__main__":
    batch_size = 8
    
    sampler = SubsetRandomSampler(range(1000))
    
    dataset_train=LeukemiaLoader(root='new_dataset/train/',mode='train')
    loader_train=DataLoader(dataset=dataset_train, sampler = sampler, batch_size=batch_size,shuffle=False,num_workers=4)
   
    dataset_valid=LeukemiaLoader(root='new_dataset/valid/',mode='valid')
    loader_valid=DataLoader(dataset=dataset_valid, sampler = sampler,batch_size=batch_size,shuffle=False,num_workers=4)

    dataset_test=LeukemiaLoader(root='new_dataset/test/',mode='test')
    loader_test=DataLoader(dataset=dataset_test,sampler = sampler,batch_size=batch_size,shuffle=False,num_workers=4)
    

    
    neural_network_18 = ResNet_18()
    res18_train,res18_test= train_and_test(ResNet_18, loader_train,loader_valid)
    print(f"resnet18 max acc：{max(res18_train)}")
    print(f"resnet18_test max acc：{max(res18_test)}")
    plot(res18_train,res18_test)
    FILE = 'resnet_18_state_dict.pt'
    torch.save(neural_network_18.state_dict(), FILE)

    neural_network_50 = ResNet_50()
    res50_train,res50_test= train_and_test(ResNet_50, loader_train,loader_valid)
    print(f"resnet50_train max acc：{max(res50_train)}")
    print(f"resnet50_test max acc：{max(res50_test)}")
    plot(res50_train,res50_test)
    FILE = 'resnet_50_state_dict.pt'
    torch.save(neural_network_50.state_dict(), FILE)

    neural_network_152 = ResNet_152()
    res152_train,res152_test= train_and_test(ResNet_152, loader_train,loader_valid)
    print(f"resnet152_train max acc：{max(res152_train)}")
    print(f"resnet152_test max acc：{max(res152_test)}")
    plot(res152_train,res152_test)
    FILE = 'resnet_152_state_dict.pt'
    torch.save(neural_network_152.state_dict(), FILE)

> Found 7995 images...
> Found 1599 images...
> Found 1067 images...
Epoch 1
Train Loss: 0.0031, Train Accuracy: 67.43%
Test Loss: 0.0354, Test Accuracy: 36.51%
Epoch 2
Train Loss: 0.0030, Train Accuracy: 67.76%
Test Loss: 0.0144, Test Accuracy: 71.05%
Epoch 3
Train Loss: 0.0030, Train Accuracy: 72.04%
Test Loss: 0.0142, Test Accuracy: 65.79%
Epoch 4
Train Loss: 0.0026, Train Accuracy: 74.34%
Test Loss: 0.0439, Test Accuracy: 39.80%
Epoch 5
Train Loss: 0.0024, Train Accuracy: 75.66%
Test Loss: 0.0166, Test Accuracy: 66.45%
Epoch 6
Train Loss: 0.0027, Train Accuracy: 74.01%
Test Loss: 0.0142, Test Accuracy: 73.68%
Epoch 7
Train Loss: 0.0023, Train Accuracy: 78.29%
Test Loss: 0.0280, Test Accuracy: 70.72%
Epoch 8
Train Loss: 0.0023, Train Accuracy: 76.32%
Test Loss: 0.0322, Test Accuracy: 49.67%
Epoch 9
Train Loss: 0.0024, Train Accuracy: 78.29%
Test Loss: 0.0103, Test Accuracy: 81.58%
Epoch 10
Train Loss: 0.0023, Train Accuracy: 76.64%
Test Loss: 0.0112, Test Accuracy: 78.62%
Epoch 11
T

NameError: name 'model' is not defined